<a href="https://colab.research.google.com/github/dina-lab3D/ContactNet/blob/master/ContactNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dina-lab3D/tutorials/blob/main/PatchDock/docking.ipynb)
#Antigen-antibody docking tutorial
<img src="https://drive.google.com/uc?export=view&id=1J6FA12dZKcf7ydSQdnFo8b7bwi-NehB5" width="300">
<img src="https://drive.google.com/uc?export=view&id=1J83QaZWoK1Xi2-dqt7o8jdImcJ8HDJf-" width="150">

In this tutorial we use a docking software to dock a modeled antibody to an antigen. We will work in 2 steps:


1. Docking using PatchDock
2. Scoring using interface scoring function ContactNet



In [1]:
#@title Download docking software - PatchDock
!wget https://github.com/dina-lab3D/tutorials/raw/main/PatchDock/pd.zip
!unzip pd.zip

--2024-01-06 16:49:30--  https://github.com/dina-lab3D/tutorials/raw/main/PatchDock/pd.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dina-lab3D/tutorials/main/PatchDock/pd.zip [following]
--2024-01-06 16:49:30--  https://raw.githubusercontent.com/dina-lab3D/tutorials/main/PatchDock/pd.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3367699 (3.2M) [application/zip]
Saving to: ‘pd.zip’

pd.zip              100%[===================>]   3.21M  13.3MB/s    in 0.2s    

2024-01-06 16:49:32 (13.3 MB/s) - ‘pd.zip’ saved [3367699/3367699]

Archive:  pd.zip
  inflating: PatchDock/buildParams.p

In [2]:
#@title Download and install ContactNet
%cd /content/
!git clone https://github.com/dina-lab3D/ContactNet --quiet
!pip install tensorflow_addons
!pip install wandb
%cd ContactNet/src/ComplexDistogram/
!cmake .
!make


/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
/content/ContactNet/src/ComplexDistogram
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done (1.2s)
-- Generati

In [3]:
#@title Prepare input for docking in the example folder
%cd /content/
%mkdir example
!cp /content/ContactNet/example/*.pdb /content/example/

/content


In [5]:
#@title run PatchDock
%cd /content/example/
# dock the antibody model to the antigen, final clustering of 4A, "AA" tells PatchDock to use antigen-antibody docking and focus on nanobody CDRs
! ../PatchDock/buildParams.pl 1kip_Ag.pdb 1kip_AF2_Ab.pdb 4.0 AA
! ../PatchDock/patch_dock.Linux params.txt docking.res
# generate top 10 by geometric score
! ../PatchDock/transOutput.pl docking.res 1 10

Number of vdW radius types: 15
Total number of atoms: 1001
Calculation of surface of 1001 atoms, with surface point density 10 and probe radius 1.8
150 yon and 471 victim probes
394 saddle and 255 concave surface points removed during non-symmetry overlapping reentrant surface removal
16941 contact, 34062 reentrant, and 51003 total surface points
Contact area: 1772.13; reentrant area: 3470.58; total area: 5242.71
    In Layer 0 out of 13 curr->size() 23509
    In Layer 1 out of 13 curr->size() 62078
    In Layer 2 out of 13 curr->size() 62725
    In Layer 3 out of 13 curr->size() 66074
    In Layer 4 out of 13 curr->size() 70997
    In Layer 5 out of 13 curr->size() 78906
    In Layer 6 out of 13 curr->size() 88530
    In Layer 7 out of 13 curr->size() 97761
    In Layer 8 out of 13 curr->size() 106140
    In Layer 9 out of 13 curr->size() 114146
    In Layer 10 out of 13 curr->size() 121789
    In Layer 11 out of 13 curr->size() 129884
    In Layer 12 out of 13 curr->size() 138743
   

In [ ]:
#@title run ContactNet scoring
!python3 /content/ContactNet/data_maker.py 1kip_Ag.pdb 1kip_AF2_Ab.pdb docking.res --trans_num 1000
!python3 /content/ContactNet/predict_prot.py 1kip_Ag.pdb 1kip_AF2_Ab.pdb --trans_num 1000

Creating ./1kip_Ag.pdb.dssp from ./1kip_Ag.pdb
 !!! Residue LEU  129 C has  5 instead of expected   4 sidechain atoms.
     last sidechain atom name is  OXT
     calculated solvent accessibility includes extra atoms !!!

 !!! Error in SSBOND data records !!!
Creating ./1kip_AF2_Ab.pdb.dssp from ./1kip_AF2_Ab.pdb
 !!! HEADER-card missing !!!
 !!! COMPOUND-card missing !!!
 !!! SOURCE-card missing !!!
 !!! AUTHOR-card missing !!!
 !!! Additional ssbond found between residues CYS   23 A and CYS   88 A !!!

 !!! Additional ssbond found between residues CYS   22 B and CYS   95 B !!!

/content/ContactNet/src/ComplexDistogram/complex_distogram_maker ../1kip_Ag.pdb ../1kip_AF2_Ab.pdb ../docking.res 1000
/content/ContactNet/src/ComplexDistogram/complex_distogram_maker ../1kip_Ag.pdb ../1kip_AF2_Ab.pdb ../docking.res 1000 
Mol1 CA size: 129
Mol2 CA size: 223
saving distogram: ../1kip_Ag_self_distogram.npy
saving distogram: ../1kip_AF2_Ab_self_distogram.npy
Invalid input. Please try again!
Invali

In [ ]:
#@title plot scores vs. RMSDs (funnel)

# create a file with interface rmsds and the scores
!grep "|" docking.res | awk '{print $1" "$4} ' | tr '(' ' ' | tr ')' ' ' > rmsds
!sed -i 's/|/rmsd/' rmsds
!cut -f2 evaluation > cnet
!paste -d ' ' rmsds cnet | head -1001 > all_scores.txt

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from plotnine import *
from scipy import stats
from scipy.interpolate import make_interp_spline

df = pd.read_csv("all_scores.txt", delimiter=r"\s+")
#print (df)
sns.set_theme(style="whitegrid")
sns.set(font_scale = 2)
sns.set_style("whitegrid")

fig1 = plt.figure(1, figsize=(6, 4), dpi=100)

sns.scatterplot(data = df,x="rmsd", y="score")
plt.show()
plt.savefig('scatter.png')

# reverse ContactNet scores (now lower is better)
#df["score"] = -df["score"]

# select best model
best_model = int(df["#"].iloc[df["score"].argmax()])
print(best_model)




In [ ]:
!pip install py3Dmol
#@title Display 3D structure {run: "auto"}
import py3Dmol
import glob

#@title Display best scoring model
! /content/PatchDock/transOutput.pl docking.res $best_model $best_model
with open("1kip_AF2_Ab.pdb") as ifile:
    true = "".join([x for x in ifile])
with open(f"docking.res.{best_model}.pdb") as ifile:
    predicted = "".join([x for x in ifile])


r,g,b = 255,0,0
print(f"\033[38;2;{r};{g};{b}mPredicted model \033[38;2;255;255;255m")
r,g,b = 0,0,255
print(f"\033[38;2;{r};{g};{b}mNative structure \033[38;2;255;255;255m")
view = py3Dmol.view(width=500, height=500)
view.addModelsAsFrames(predicted)
view.setStyle({'model': 0, "chain": "C"}, {"cartoon": {'arrows':True, 'color': 'gray'}})
view.setStyle({'model': 0, "chain": "A"}, {"cartoon": {'arrows':True, 'color': 'red'}})
view.setStyle({'model': 0, "chain": "B"}, {"cartoon": {'arrows':True, 'color': 'red'}})
view.addModelsAsFrames(true)
view.setStyle({'model': 1}, {"cartoon": {'arrows':True, 'color': 'blue'}})
view.zoomTo()
view.show()
